In [1]:
# creating results after creating new fxn that utilizes the ratio of the number 
# of components, instead of the number of proteins in a cluster

In [2]:
# importing libraries

# standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# library used for functional enrichment analysis
from func_e.FUNC_E import FUNC_E  # a method that can do funtional enrichment analysis!
import func_e.vocabs.all as vocabs

In [4]:
# following code is necessary as some functions used in the funcE class produce future warnings but i can't fix it as it is not my code
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
# importing the custom classes i built
from matrix_class import ProteinMatrix # ppi matrix 
from cluster_class import AllClusters # dictionary to hold all clusters (in form number of cluster : list of proteins in that cluster)
from degreelist_class import DegreeList # creates a list of all proteins in order of their degree

In [6]:
# helper functions for setting up program
from recipe_utils import initialize_matrix_clusters_degreelist
from recipe_utils import find_clusters_and_proteins_together

# helper functions for functional enrichment
from recipe_utils import print_querylist_of_clusters_to_file
from recipe_utils import print_protein_background_to_file
from recipe_utils import create_term_mapping_list
from recipe_utils import get_initialized_fe

In [7]:
matrix, clusters, degreelist = initialize_matrix_clusters_degreelist("data/interactions/1_ppi_anonym_v2.txt", "data/clusters/1_clusters_s100.txt")

In [8]:
qualifying_clusters, qualifying_proteins = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2,use_sqrt=True, protein_ratio=1, protein_constant=0)

# cluster const = 2 -> must have at least 2 componenets



In [ ]:
print(qualifying_proteins)

In [11]:
qualifying_clusters5, qualifying_proteins5 = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2,use_sqrt=True, protein_ratio=.5, protein_constant=0)



In [12]:
print(qualifying_proteins5)


{}


In [10]:
qualifying_clusters3, qualifying_proteins3 = find_clusters_and_proteins_together(matrix, clusters, degreelist, cluster_ratio=0, cluster_constant=2,use_sqrt=True, protein_ratio=.25, protein_constant=0)
print(qualifying_proteins3)


13920 not in degreelist dictionary
{7: [17391, 17376, 17351, 17336, 17319, 17289, 17277, 17276, 17184, 17146, 17132, 17121, 17079, 17072, 17064, 17061, 17042, 16970, 16964, 16962, 16950, 16931, 16919, 16891, 16876, 16867, 16850, 16849, 16824, 16815, 16787, 16782, 16753, 16748, 16735, 16730, 16717, 16713, 16707, 16543, 16520, 16500, 16470, 16436, 16419, 16400, 16377, 16374, 16354, 16319, 16317, 16249, 16227, 16223, 16163, 16136, 16065, 16062, 16038, 15979, 15972, 15950, 15878, 15877, 15840, 15811, 15800, 15781, 15776, 15756, 15748, 15723, 15722, 15710, 15653, 15631, 15617, 15616, 15594, 15577, 15533, 15524, 15523, 15512, 15510, 15507, 15495, 15489, 15450, 15437, 15419, 15385, 15375, 15344, 15271, 15263, 15256, 15251, 15234, 15200, 15193, 15187, 15175, 15111, 15074, 15049, 15037, 15032, 14980, 14965, 14959, 14938, 14904, 14885, 14873, 14861, 14849, 14845, 14836, 14827, 14780, 14777, 14758, 14733, 14725, 14707, 14705, 14681, 14668, 14665, 14651, 14650, 14568, 14549, 14548, 14537, 14508, 1

In [ ]:
# use functions to print info to files: (annoying, but need to get info into format used for funcE package)


# 1) a background protein list to be used for functional enrichment analysis
background_proteins_filepath = "demo_data/background_proteinlist.txt"
print_protein_background_to_file(matrix, background_proteins_filepath) 

# 2) file containing protein to go term
term_mapping_filepath = "demo_data/term_mapping.txt"
create_term_mapping_list("demo_data/dream3_go_results.tsv", term_mapping_filepath)

# 3) not a file, but need to establish a background of all GO terms! (to be used for functional enrichment analysis)
background_go_terms = vocabs.getTerms(['GO'])

In [ ]:
# continue printing info to files to be used in functional enrichment analysis:

# 4) print original clusters to a file
og_query_filepath = 'demo_data/original_clusters.txt'
print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(),og_query_filepath)

# 5) print updated clusters to a file (just append the function to include the dictionary of qualifying proteins)
updated_query_filepath = 'demo_data/updated_clusters.txt'
print_querylist_of_clusters_to_file(clusters, clusters.get_all_cluster_labels(), updated_query_filepath, qualifying_proteins)

In [ ]:
original_fe = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)

original_fe.importFiles({'query': og_query_filepath })
original_fe.run(cluster=False)

In [ ]:
# do functional enrichment on updated clusters!!

updated_fe_all_proteins = get_initialized_fe(background_proteins_filepath, term_mapping_filepath, termlist = background_go_terms)

updated_fe_all_proteins.importFiles({'query': updated_query_filepath })
updated_fe_all_proteins.run(cluster=False)

In [ ]:
# print data by the numbers!
print(f"number of clusters that were updated with proteins: {len(qualifying_proteins.keys())}\n")

# clusters that are functionally enriched
print(f"number of functionally enriched clusters (modules) out of {len(qualifying_proteins.keys())} original clusters: {original_fe.enrichment['Module'].nunique()}")
print(f"number of functionally enriched clusters after addition of all proteins: {updated_fe_all_proteins.enrichment['Module'].nunique()}")

print(f"")


# significant P-values!!!
print(f"number of significant pvals in original clusters before updating: {original_fe.enrichment['Fishers_pvalue'].count()}")
print(f"number of significant pvals in clusters with all proteins added: {updated_fe_all_proteins.enrichment['Fishers_pvalue'].count()}")

number of clusters that were updated with proteins: 12

number of functionally enriched clusters (modules) out of 12 original clusters: 155
number of functionally enriched clusters after addition of all proteins: 155

number of significant pvals in original clusters before updating: 405
number of significant pvals in clusters with all proteins added: 411
